In [1]:
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import os, time, uuid
from dotenv import load_dotenv
from pathlib import Path

dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

True

In [2]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials

In [3]:
TRAINING_ENDPOINT = os.getenv("CUSTOM_VISION_TRAINING_ENDPOINT")
training_key = os.getenv("CUSTOM_VISION_TRAINING_KEY")
training_resource_id = os.getenv("CUSTOM_VISION_TRAINING_RESOURCE_ID")

In [4]:
PREDICTION_ENDPOINT = os.getenv("CUSTOM_VISION_PREDICTION_ENDPOINT")
prediction_key = os.getenv("CUSTOM_VISION_PREDICTION_KEY")
prediction_resource_id = os.getenv("CUSTOM_VISION_PREDICTION_RESOURCE_ID")

In [5]:
training_credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})


trainer = CustomVisionTrainingClient(TRAINING_ENDPOINT, training_credentials)

In [7]:
# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print ("Your Object Detection Training project has been created. Please move on.")
project_name = "Lighter detection"
project = trainer.create_project(project_name, domain_id=obj_detection_domain.id)

Your Object Detection Training project has been created. Please move on.


In [7]:
projects = trainer.get_projects()
for project in projects:
    print(f"Project Name: {project.name}")
    print(f"Project ID: {project.id}")
    print(f"Description: {project.description}")
    print(f"Domain: {project.settings.domain_id}")
    print(f"Created: {project.created}\n")

Project Name: Lighter detection
Project ID: e5194cc9-d361-4ef1-84ae-f1f6aa816152
Description: 
Domain: a27d5ca5-bb19-49d8-a70a-fec086c47f5b
Created: 2024-09-08 18:48:40.866000+00:00



In [8]:
lighter_tag = trainer.create_tag(project.id, "Lighter")

In [8]:
iteration.as_dict()

NameError: name 'iteration' is not defined

In [9]:
iteration_list = trainer.get_iterations(project.id)
train_iteration = []
for iteration_item in iteration_list:
    print(iteration_item)
    train_iteration.append(iteration_item)
    print("=====================================")

{'additional_properties': {}, 'id': '049d6d06-b25e-4f1b-879e-b41b1adf222c', 'name': 'Iteration 4', 'status': 'Completed', 'created': datetime.datetime(2024, 9, 9, 8, 11, 15, 243000, tzinfo=<isodate.tzinfo.Utc object at 0x000001C3A6641F50>), 'last_modified': datetime.datetime(2024, 9, 9, 11, 25, 42, 39000, tzinfo=<isodate.tzinfo.Utc object at 0x000001C3A6641F50>), 'trained_at': datetime.datetime(2024, 9, 9, 11, 25, 42, 35000, tzinfo=<isodate.tzinfo.Utc object at 0x000001C3A6641F50>), 'project_id': 'e5194cc9-d361-4ef1-84ae-f1f6aa816152', 'exportable': True, 'exportable_to': None, 'domain_id': 'a27d5ca5-bb19-49d8-a70a-fec086c47f5b', 'classification_type': None, 'training_type': 'Regular', 'reserved_budget_in_hours': 0, 'training_time_in_minutes': 1, 'publish_name': None, 'original_publish_resource_id': None, 'custom_base_model_info': None, 'training_error_details': None}
{'additional_properties': {}, 'id': '35318f6f-2e3a-4ce0-9983-179b1a9790e3', 'name': 'Iteration 3', 'status': 'Completed

In [10]:
train_iteration[1].id, iteration_list[1].id

('35318f6f-2e3a-4ce0-9983-179b1a9790e3',
 '35318f6f-2e3a-4ce0-9983-179b1a9790e3')

In [11]:
model_perf = trainer.get_iteration_performance(project.id, iteration_list[1].id)

In [12]:
model_perf.as_dict()

{'per_tag_performance': [{'id': '9976db76-6af8-4505-bd0f-a6783cf8de8e',
   'name': 'Lighter',
   'precision': 1.0,
   'precision_std_deviation': 0.0,
   'recall': 0.13333334,
   'recall_std_deviation': 0.0,
   'average_precision': 0.51109034}],
 'precision': 1.0,
 'precision_std_deviation': 0.0,
 'recall': 0.13333334,
 'recall_std_deviation': 0.0,
 'average_precision': 0.51109034}

In [13]:
#Publish the model
# iteration_id = model_perf.as_dict()['per_tag_performance'][0]['id']
iteration_id = iteration_list[1].id
iteration_id

'35318f6f-2e3a-4ce0-9983-179b1a9790e3'

In [14]:
publish_iteration_name = "lighter-object-detection-custom"

In [69]:
trainer.publish_iteration(project.id, iteration_id, publish_iteration_name, prediction_resource_id)
print ("Done!")

CustomVisionErrorException: Iteration is already published as: lighter-object-detection-custom

In [36]:
#Perform Prediction

In [15]:
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(PREDICTION_ENDPOINT, prediction_credentials)

In [16]:
local_image_path = 'lighter_test_images'
file_name = "lighter_test_set_1of5.jpg"


In [23]:
def perform_prediction(image_file_name):
    with open(os.path.join(local_image_path, image_file_name), "rb") as image_contents:
        results = predictor.detect_image(project.id, publish_iteration_name, image_contents.read())
        
        # Sort predictions by probability in descending order
        sorted_predictions = sorted(results.predictions, key=lambda p: p.probability, reverse=True)
        
        # Get the top 56 predictions
        top_5_predictions = sorted_predictions[:5]
        
        # Display the top 56 results
        for prediction in top_5_predictions:
            print("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))

# Example usage
perform_prediction("lighter_test_set_1of5.jpg")

	Lighter: 80.18%
	Lighter: 66.62%
	Lighter: 21.57%
	Lighter: 15.25%
	Lighter: 9.49%


In [24]:
lighter_images = os.listdir('lighter_test_images')
lighter_images

['lighter_test_set_1of5.jpg',
 'lighter_test_set_2of5.jpg',
 'lighter_test_set_3of5.jpg',
 'lighter_test_set_4of5.jpg',
 'lighter_test_set_5of5.jpg']

In [29]:
import os

def perform_prediction_on_multiple_images(image_folder):
    image_list = os.listdir(image_folder)
    for image_file_name in image_list:
        with open(os.path.join(image_folder, image_file_name), "rb") as image_contents:
            results = predictor.detect_image(project.id, publish_iteration_name, image_contents.read())
            
            # Sort predictions by probability in descending order
            sorted_predictions = sorted(results.predictions, key=lambda p: p.probability, reverse=True)
            
            # Get the top 5 predictions
            top_5_predictions = sorted_predictions[:5]
            
            # Display the top 5 predictions with bounding box details
            for prediction in top_5_predictions:
                print("\t" + prediction.tag_name + 
                      ": {0:.2f}% bbox.left = {1:.2f}, bbox.top = {2:.2f}, bbox.width = {3:.2f}, bbox.height = {4:.2f}".format(
                          prediction.probability * 100, 
                          prediction.bounding_box.left, 
                          prediction.bounding_box.top, 
                          prediction.bounding_box.width, 
                          prediction.bounding_box.height))
            
            print(f"=================== Image name => {image_file_name} ======================================================")




In [30]:
perform_prediction_on_multiple_images(local_image_path)

	Lighter: 80.18% bbox.left = 0.33, bbox.top = 0.46, bbox.width = 0.22, bbox.height = 0.24
	Lighter: 66.62% bbox.left = 0.33, bbox.top = 0.12, bbox.width = 0.50, bbox.height = 0.77
	Lighter: 21.57% bbox.left = 0.47, bbox.top = 0.03, bbox.width = 0.49, bbox.height = 0.66
	Lighter: 15.25% bbox.left = 0.51, bbox.top = 0.24, bbox.width = 0.49, bbox.height = 0.69
	Lighter: 9.49% bbox.left = 0.22, bbox.top = 0.19, bbox.width = 0.41, bbox.height = 0.76
=================== Image name => lighter_test_set_1of5.jpg ======================================================
	Lighter: 64.52% bbox.left = 0.26, bbox.top = 0.20, bbox.width = 0.49, bbox.height = 0.54
	Lighter: 21.35% bbox.left = 0.18, bbox.top = 0.04, bbox.width = 0.65, bbox.height = 0.53
	Lighter: 19.44% bbox.left = 0.31, bbox.top = 0.55, bbox.width = 0.26, bbox.height = 0.28
	Lighter: 11.12% bbox.left = 0.00, bbox.top = 0.01, bbox.width = 0.11, bbox.height = 0.12
	Lighter: 6.48% bbox.left = 0.32, bbox.top = 0.37, bbox.width = 0.48, bbox.h

CustomVisionErrorException: Operation returned an invalid status code 'Too Many Requests'

In [34]:
#Export Model as tensorflow
platform = "TensorFlow"
flavor = "TensorFlowLite"

In [35]:
export_process = trainer.export_iteration(project.id, iteration_id, platform, flavor, raw=True)

In [36]:
print(export_process.output)

{'additional_properties': {}, 'platform': 'TensorFlow', 'status': 'Exporting', 'download_uri': None, 'flavor': 'TensorFlowLite', 'newer_version_available': False}


In [37]:
print(export_process.output.status)

Exporting


In [39]:
### Code snippet is from Azure SDK and Documentation
### https://docs.microsoft.com/en-us/azure/cognitive-services/custom-vision-service/export-programmatically
### This step may take long time 
while (export_process.output.status == "Exporting"):
    print ("Waiting 10 seconds...")
    time.sleep(10)
    exports = trainer.get_exports(project.id, iteration_id)
    for e in exports:
        if e.platform == export_process.output.platform and e.flavor == export_process.output.flavor:
            export = e
            break
    print("Export status is: ", export_process.output.status)

Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export st

KeyboardInterrupt: 

In [40]:
print(export_process.output.status)

Exporting
